### 0. Ścieżki

In [1]:


### Ścieżki wejścia/wyjścia dla modułu "Quality Metrics"

from pathlib import Path

PROJECT_ROOT = Path(".").resolve()

# katalog z obrazami
IMAGE_ROOT = PROJECT_ROOT / "inputs"   # jeżeli masz inną ścieżkę, zostaw swoją
print("IMAGE_ROOT:", IMAGE_ROOT, "| istnieje:", IMAGE_ROOT.exists())

# katalog artefaktów jakości (opcjonalny: miniatury, wykresy)
OUTPUT_DIR = PROJECT_ROOT / "outputs" /"csv"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# wspólny katalog na CSV całego pipeline'u
OUTPUT_CSV_DIR = PROJECT_ROOT / "outputs" / "csv"
OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)

# finalny CSV z metrykami jakości
QUALITY_CSV = OUTPUT_CSV_DIR / "quality_metrics.csv"

print("QUALITY_CSV:", QUALITY_CSV)

IMAGE_ROOT: /Users/olga/MetaLogic/inputs | istnieje: True
QUALITY_CSV: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv


### 01 – Importy + ścieżki + lista obrazów

In [2]:
# %%
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image

DIR_DATA = Path("inputs")

def list_images(dir_path: Path):
    exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
    return sorted([p for p in dir_path.iterdir() if p.suffix.lower() in exts])

image_paths = list_images(DIR_DATA)
print("Liczba obrazów:", len(image_paths))

Liczba obrazów: 134


### 02 – Metryki jakości

In [3]:
# %%
def load_cv2_gray(path: Path):
    """
    Wczytuje obraz w skali szarości przez OpenCV.

    Zwraca:
    - tablicę uint8 (H, W) jeśli się udało,
    - None, jeśli OpenCV nie potrafi wczytać pliku (np. nietypowy TIFF/WEBP).
    """
    try:
        img_gray = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    except Exception as e:
        print(f"[QUALITY] Błąd odczytu pliku {path.name}: {e}")
        return None

    if img_gray is None:
        print(f"[QUALITY] OpenCV nie wczytał pliku (None): {path.name}")
        return None

    return img_gray


def blur_score(img_gray):
    return float(cv2.Laplacian(img_gray, cv2.CV_64F).var())


def contrast_score(img_gray):
    return float(img_gray.std())


def brightness_score(img_gray):
    return float(img_gray.mean())


def compute_quality(path: Path):
    """
    Liczy metryki jakości dla pojedynczego obrazu.

    Zwraca dict:
    - file_path, blur, contrast, brightness
    lub None, jeśli obraz nie został wczytany.
    """
    img_gray = load_cv2_gray(path)
    if img_gray is None:
        return None

    return {
        "file_path": str(path),
        "blur": blur_score(img_gray),
        "contrast": contrast_score(img_gray),
        "brightness": brightness_score(img_gray),
    }

### 03 – Batch

In [4]:
# %%
records = []

for i, p in enumerate(image_paths, start=1):
    print(f"[{i}/{len(image_paths)}] {p.name}")
    rec = compute_quality(p)
    if rec:
        records.append(rec)

df_quality = pd.DataFrame(records)
df_quality



[1/134] 0000.jpg
[2/134] 000202.jpg
[3/134] 000238.jpg
[4/134] 0003.jpg
[5/134] 000345.jpg
[6/134] 000346.jpg
[7/134] 000388.jpg
[8/134] 000391.jpg
[9/134] 0004.jpg
[10/134] 000406.jpg
[11/134] 0006 2.jpg
[12/134] 0006.jpg
[13/134] 0009.jpg
[14/134] 0022.jpg
[15/134] 0034.jpg
[16/134] 0043.jpg
[17/134] 0044.jpg
[18/134] 0074.jpg
[19/134] 0075.jpg
[20/134] 0077.jpg
[21/134] 0079.jpg
[22/134] 0106.jpg
[23/134] 0109.jpg
[24/134] 011.jpg
[25/134] 0111.jpg
[26/134] 0112.jpg
[27/134] 0114.jpg
[28/134] 0115 2.jpg
[29/134] 0115.jpg
[30/134] 0116.jpg
[31/134] 0117.jpg
[32/134] 0119.jpg
[33/134] 0120.jpg
[34/134] 0121.jpg
[35/134] 0124 2.jpg
[36/134] 0124.jpg
[37/134] 0125.jpg
[38/134] 0126.jpg
[39/134] 0127.jpg
[40/134] 0128.jpg
[41/134] 0135.jpg
[42/134] 0136.jpg
[43/134] 0138.jpg
[44/134] 0143.jpg
[45/134] 0145.jpg
[46/134] 015_Kobiety z ziołami do święcenia - Bukowina Tatrzańska - 000836s.jpg
[47/134] 0161.jpg
[48/134] 0165.jpg
[49/134] 0171.jpg
[50/134] 0174.jpg
[51/134] 0175.jpg
[52/134] 0

,file_path,blur,contrast,brightness
0,inputs/0000.jpg,451.646616,71.527650,130.343196
1,inputs/000202.jpg,346.484591,75.248329,105.953739
2,inputs/000238.jpg,449.996789,79.478098,123.166496
3,inputs/0003.jpg,427.360351,76.946272,132.402162
4,inputs/000345.jpg,432.544893,66.824435,112.437095
...,...,...,...,...
129,inputs/wO9k9lBaHR0cHM6Ly9vY2RuLmV1L3B1bHNjbXMv...,1963.113196,58.770740,146.100972
130,inputs/z00000589.jpg,2486.081988,85.303519,125.856201
131,"inputs/z21526004Q,Stare-Miasto.jpg",4430.857114,70.950202,152.489268
132,"inputs/z29438216Q,1994-r--Gorniki-k--Bytomia--...",3546.939419,71.684726,120.614612


### CSV

In [5]:
# %%
out_path = Path("outputs/quality_metrics.csv")
df_quality.to_csv(QUALITY_CSV, index=False)
print("Zapisano metryki jakości do:", QUALITY_CSV)

Zapisano metryki jakości do: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv


### 1. Sortowanie zdjęć po ostrości (blur)

In [6]:
# %%
"""
Sortowanie zdjęć po ostrości (blur).

Tworzy dwie tabele:
- df_sharpest  – zdjęcia od najostrzejszych
- df_blurriest – zdjęcia od najbardziej rozmytych
"""

df_sharpest = df_quality.sort_values("blur", ascending=False).reset_index(drop=True)
df_blurriest = df_quality.sort_values("blur", ascending=True).reset_index(drop=True)

print("Najostrzejsze 10 zdjęć:")
df_sharpest.head(10)

Najostrzejsze 10 zdjęć:


,file_path,blur,contrast,brightness
0,"inputs/z21526004Q,Stare-Miasto.jpg",4430.857114,70.950202,152.489268
1,inputs/0117.jpg,3991.757403,84.260545,141.577361
2,"inputs/z29438216Q,1994-r--Gorniki-k--Bytomia--...",3546.939419,71.684726,120.614612
3,inputs/0119.jpg,3535.061055,55.277084,126.753168
4,inputs/124221_Skrzyzownie_ulicy_Dolnej_Panny_M...,3464.681971,57.680943,112.795868
5,inputs/Nysa_521M_MPK_Kraków.jpg,3278.729839,70.405921,139.520412
6,inputs/489020068_1120771796757226_461945449554...,2709.633593,62.879095,124.793939
7,inputs/syrena_kombi_coupe.jpg,2693.288020,59.149574,175.146552
8,inputs/z00000589.jpg,2486.081988,85.303519,125.856201
9,inputs/doba_pl_38237-be832b0bcf8291fd11ce4848b...,2469.566168,51.082555,141.479161


In [7]:
# %%
print("Najbardziej rozmyte 10 zdjęć:")
df_blurriest.head(10)

Najbardziej rozmyte 10 zdjęć:


,file_path,blur,contrast,brightness
0,inputs/Screenshot 2025-11-20 at 09.50.21.png,9.139253,69.477708,132.475647
1,inputs/0226.jpg,41.226857,95.203951,172.202915
2,inputs/015_Kobiety z ziołami do święcenia - Bu...,50.130747,57.938554,155.381424
3,inputs/547483793_1255800123254392_484096360664...,60.873279,48.503155,128.632422
4,inputs/487199256_1115723353928737_484056380111...,74.427954,67.593542,146.375370
5,inputs/0188.jpg,74.551545,57.136059,180.875610
6,inputs/0136.jpg,77.113161,61.731172,135.309397
7,inputs/000388.jpg,84.161698,74.662799,61.974815
8,inputs/default-4.jpg,86.640338,83.075988,80.469344
9,inputs/0125.jpg,105.324313,58.087298,173.330123


### 2. Filtrowanie „najgorszych” jakościowo (np. 10% najbardziej rozmytych)

In [8]:
# %%
"""
Wybór najgorszych jakościowo zdjęć pod względem rozmycia.

Wyznacza próg jako 10. percentyl wartości 'blur'
i zwraca wszystkie zdjęcia poniżej tego progu.
"""

blur_threshold = df_quality["blur"].quantile(0.10)
print("Próg blur (10% najbardziej rozmyte):", blur_threshold)

df_worst_blur = df_quality[df_quality["blur"] <= blur_threshold].copy()
df_worst_blur = df_worst_blur.sort_values("blur", ascending=True).reset_index(drop=True)

print("Liczba zdjęć uznanych za najbardziej rozmyte:", len(df_worst_blur))
df_worst_blur

Próg blur (10% najbardziej rozmyte): 111.03252606582004
Liczba zdjęć uznanych za najbardziej rozmyte: 14


,file_path,blur,contrast,brightness
0,inputs/Screenshot 2025-11-20 at 09.50.21.png,9.139253,69.477708,132.475647
1,inputs/0226.jpg,41.226857,95.203951,172.202915
2,inputs/015_Kobiety z ziołami do święcenia - Bu...,50.130747,57.938554,155.381424
3,inputs/547483793_1255800123254392_484096360664...,60.873279,48.503155,128.632422
4,inputs/487199256_1115723353928737_484056380111...,74.427954,67.593542,146.375370
5,inputs/0188.jpg,74.551545,57.136059,180.875610
6,inputs/0136.jpg,77.113161,61.731172,135.309397
7,inputs/000388.jpg,84.161698,74.662799,61.974815
8,inputs/default-4.jpg,86.640338,83.075988,80.469344
9,inputs/0125.jpg,105.324313,58.087298,173.330123


In [9]:
### X. Progi i flagi jakości

"""
Dodanie kolumn flag jakościowych na podstawie prostych progów.

df_quality      – DataFrame z metrykami jakości (blur / brightness / contrast)
BLUR_MIN_GOOD   – minimalna wartość blur uznawana za „wystarczającą ostrość”
BRIGHTNESS_MIN  – dolna granica akceptowalnej jasności
BRIGHTNESS_MAX  – górna granica akceptowalnej jasności
CONTRAST_MIN    – dolna granica akceptowalnego kontrastu
CONTRAST_MAX    – górna granica akceptowalnego kontrastu
"""

import pandas as pd

BLUR_MIN_GOOD = 50.0
BRIGHTNESS_MIN = 0.25
BRIGHTNESS_MAX = 0.85
CONTRAST_MIN = 0.15
CONTRAST_MAX = 0.85


def classify_blur(value: float) -> str:
    """
    Zwraca prostą flagę jakości ostrości.

    value – wartość metryki blur dla pojedynczego obrazu
    """
    if pd.isna(value):
        return "unknown"
    return "ok" if value >= BLUR_MIN_GOOD else "too_blurry"


def classify_brightness(value: float) -> str:
    """
    Zwraca prostą flagę jakości jasności.

    value – wartość metryki brightness dla pojedynczego obrazu
    """
    if pd.isna(value):
        return "unknown"
    if value < BRIGHTNESS_MIN:
        return "too_dark"
    if value > BRIGHTNESS_MAX:
        return "too_bright"
    return "ok"


def classify_contrast(value: float) -> str:
    """
    Zwraca prostą flagę jakości kontrastu.

    value – wartość metryki contrast dla pojedynczego obrazu
    """
    if pd.isna(value):
        return "unknown"
    if value < CONTRAST_MIN:
        return "too_low"
    if value > CONTRAST_MAX:
        return "too_high"
    return "ok"


df_quality["flag_blur"] = df_quality["blur"].map(classify_blur)
df_quality["flag_brightness"] = df_quality["brightness"].map(classify_brightness)
df_quality["flag_contrast"] = df_quality["contrast"].map(classify_contrast)


def combine_flags(row: pd.Series) -> str:
    """
    Łączy flagi cząstkowe w jedną flagę jakości.

    row – wiersz DataFrame z kolumnami flag_blur, flag_brightness, flag_contrast
    """
    flags = {row["flag_blur"], row["flag_brightness"], row["flag_contrast"]}
    if flags <= {"ok"}:
        return "ok"
    if "unknown" in flags and flags == {"unknown"}:
        return "unknown"
    return "bad"


df_quality["quality_flag"] = df_quality.apply(combine_flags, axis=1)

# Kolumna na notatki ludzkie – tylko jeśli jeszcze nie istnieje
if "quality_notes" not in df_quality.columns:
    df_quality["quality_notes"] = ""

In [10]:
### X. Podsumowanie globalne i po folderach

"""
Tworzy podsumowania metryk jakości:
- opis statystyczny dla całego zbioru,
- agregację po folderze nadrzędnym (np. serii skanów).

df_quality – DataFrame z metrykami i flagami jakości
"""

from pathlib import Path

# Kolumny pomocnicze: nazwa pliku i folder nadrzędny (jeśli jeszcze ich nie ma)
if "file_name" not in df_quality.columns:
    df_quality["file_name"] = df_quality["file_path"].map(lambda p: Path(p).name)

if "parent_folder" not in df_quality.columns:
    df_quality["parent_folder"] = df_quality["file_path"].map(lambda p: Path(p).parent.name)

summary_global = df_quality[["blur", "brightness", "contrast"]].describe(
    percentiles=[0.05, 0.5, 0.95]
)

summary_by_folder = (
    df_quality
    .groupby("parent_folder")[["blur", "brightness", "contrast"]]
    .agg(["count", "mean", "median"])
    .sort_values(("blur", "mean"))
)

print("=== Podsumowanie globalne ===")
print(summary_global)

print("\n=== Podsumowanie po folderach (posortowane po średnim blur) ===")
print(summary_by_folder.head(20))

=== Podsumowanie globalne ===
              blur  brightness    contrast
count   134.000000  134.000000  134.000000
mean    807.351855  129.976154   63.139283
std     885.626447   25.299344   10.135021
min       9.139253   61.974815   35.507076
5%       81.694710   92.330361   47.267053
50%     469.175523  128.558141   62.746498
95%    2699.008970  176.460795   80.012063
max    4430.857114  194.690162   95.203951

=== Podsumowanie po folderach (posortowane po średnim blur) ===
               blur                         brightness              \
              count        mean      median      count        mean   
parent_folder                                                        
inputs          134  807.351855  469.175523        134  129.976154   

                          contrast                        
                   median    count       mean     median  
parent_folder                                             
inputs         128.558141      134  63.139283  62.746498  


In [11]:
### X. Eksport list obrazów „do sprawdzenia”

"""
Eksportuje:
- globalny plik z metrykami jakości,
- listę najgorszych obrazów wg blur,
- listę obrazów oznaczonych jako „bad”.

df_quality     – DataFrame z metrykami jakości
OUTPUT_CSV_DIR – katalog wyjściowy na pliki CSV (zdefiniowany wcześniej w notatniku)
"""

from pathlib import Path

QUALITY_DIR = OUTPUT_CSV_DIR  # dostosuj, jeśli używasz innej zmiennej katalogu
QUALITY_DIR.mkdir(parents=True, exist_ok=True)

quality_all_path = QUALITY_DIR / "quality_metrics.csv"
worst_blur_path = QUALITY_DIR / "quality_worst_blur.csv"
bad_overall_path = QUALITY_DIR / "quality_bad_overall.csv"

# 1) Wszystkie metryki
df_quality.to_csv(quality_all_path, index=False)

# 2) Np. 10% najgorszych obrazów po blur (jeśli blur rośnie z ostrością, sortujemy rosnąco)
k = max(1, int(0.10 * len(df_quality)))
df_worst_blur = df_quality.sort_values("blur", ascending=True).head(k)
df_worst_blur.to_csv(worst_blur_path, index=False)

# 3) Wszystkie obrazy z łączną flagą jakości „bad”
df_bad = df_quality[df_quality["quality_flag"] == "bad"].copy()
df_bad.to_csv(bad_overall_path, index=False)

print("Zapisano:")
print(f"- wszystkie metryki: {quality_all_path}")
print(f"- ~10% najbardziej problematycznych (po blur): {worst_blur_path}")
print(f"- wszystkie obrazy z quality_flag == 'bad': {bad_overall_path}")

Zapisano:
- wszystkie metryki: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv
- ~10% najbardziej problematycznych (po blur): /Users/olga/MetaLogic/outputs/csv/quality_worst_blur.csv
- wszystkie obrazy z quality_flag == 'bad': /Users/olga/MetaLogic/outputs/csv/quality_bad_overall.csv
